In [2]:
!pip install lightgbm==3.3.2
!pip install pandas==1.5
!pip install scikit-learn==1.5

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.6.0
    Uninstalling lightgbm-4.6.0:
      Successfully uninstalled lightgbm-4.6.0


  You can safely remove it manually.


     ---------------------------------------- 0.0/5.2 MB ? eta -:--:--
     -- ------------------------------------- 0.3/5.2 MB ? eta -:--:--
     ------------ --------------------------- 1.6/5.2 MB 3.4 MB/s eta 0:00:02
     -------------------------- ------------- 3.4/5.2 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 5.2/5.2 MB 6.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pandas


  error: subprocess-exited-with-error
  
  × Building wheel for pandas (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [2526 lines of output]
      <string>:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      C:\Users\user\AppData\Local\Temp\pip-build-env-laiu2hw7\overlay\Lib\site-packages\setuptools\dist.py:760: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: BSD License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize

   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/10.9 MB 9.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/10.9 MB 8.0 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/10.9 MB 7.7 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/10.9 MB 8.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/10.9 MB 8.1 MB/s eta 0:00:01
   -------------------------------------- - 10.5/10.9 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 10.9/10.9 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1


  You can safely remove it manually.


# 데이터 불러오기

In [62]:
import pandas as pd
import numpy as np

In [63]:
model_df = pd.read_csv("C:\\Users\\user\\Desktop\\중간프로젝트\\의료패널 데이터\\a.csv")
model_df.drop(["HHID","PIDWON","H_INC7"], axis=1, inplace=True)
model_df.shape

(4934, 157)

# > lightGBM

## 1. 데이터 분할

In [29]:
data = model_df.copy()

In [30]:
from sklearn.model_selection import train_test_split

# 전체 데이터를 Train / Test로 분할
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("PHI_BENEFIT", axis=1),
    data['PHI_BENEFIT'],
    random_state=42
)

# 전체 데이터를 Train / Test로 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    random_state=42
)

## 2. 모델 훈련

In [31]:
# 모델 객체 생성
from lightgbm import LGBMRegressor

LGBM = LGBMRegressor(
    objective='regression',
    max_depth=5,
    random_state=42
)

In [32]:
# 조기중단을 위한 평가 세트 설정
evals = [(X_train, y_train), (X_valid, y_valid)]

# 모델 학습
LGBM.fit(X_train, y_train,
          eval_set=evals,
          eval_metric='rmse',
          early_stopping_rounds=10,  # 10 라운드 동안 개선되지 않으면 중단
          verbose=True)  # 10 라운드마다 로그 출력

[1]	training's rmse: 1.46361	training's l2: 2.14217	valid_1's rmse: 1.51269	valid_1's l2: 2.28824
[2]	training's rmse: 1.42302	training's l2: 2.025	valid_1's rmse: 1.47853	valid_1's l2: 2.18604
[3]	training's rmse: 1.38599	training's l2: 1.92098	valid_1's rmse: 1.45112	valid_1's l2: 2.10574
[4]	training's rmse: 1.35644	training's l2: 1.83992	valid_1's rmse: 1.42601	valid_1's l2: 2.03351
[5]	training's rmse: 1.32825	training's l2: 1.76425	valid_1's rmse: 1.40595	valid_1's l2: 1.97669
[6]	training's rmse: 1.30511	training's l2: 1.70331	valid_1's rmse: 1.39012	valid_1's l2: 1.93244
[7]	training's rmse: 1.28385	training's l2: 1.64826	valid_1's rmse: 1.37597	valid_1's l2: 1.8933
[8]	training's rmse: 1.26382	training's l2: 1.59723	valid_1's rmse: 1.36558	valid_1's l2: 1.86481
[9]	training's rmse: 1.25027	training's l2: 1.56318	valid_1's rmse: 1.3542	valid_1's l2: 1.83385
[10]	training's rmse: 1.23556	training's l2: 1.5266	valid_1's rmse: 1.34548	valid_1's l2: 1.81031
[11]	training's rmse: 1.

c:\Users\user\anaconda3\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\user\anaconda3\Lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(max_depth=5, objective='regression', random_state=42)

## 3. 예측 및 평가

In [33]:
# 예측
# 학습 데이터에 대한 예측
y_pred_train = LGBM.predict(X_train)
# 테스트 데이터에 대한 예측
y_pred_test = LGBM.predict(X_test)

# 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 훈련 데이터 성능 지표 계산
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
# 테스트 데이터 성능 지표 계산
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
# 결과 출력
print("--- Training Set Performance ---")
print(f"MSE: {mse_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"R² Score: {r2_train:.4f}\n")
print("--- Test Set Performance ---")
print(f"MSE: {mse_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R² Score: {r2_test:.4f}")

--- Training Set Performance ---
MSE: 1.0333
RMSE: 1.0165
MAE: 0.7849
R² Score: 0.5473

--- Test Set Performance ---
MSE: 1.4937
RMSE: 1.2222
MAE: 0.9343
R² Score: 0.3367


## 4. grid search

In [34]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
'max_depth': [3, 4],       # 깊이 축소
'reg_lambda': [0.1, 0.3],  # 정규화 강화
'learning_rate': [0.05, 0.07, 0.1],
'n_estimators': [300, 400, 500],
'subsample': [0.8]
}


# 그리드 서치 설정
grid_search = GridSearchCV(
    estimator=LGBM,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적 결과 출력
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", -grid_search.best_score_)


# 최적 모델로 예측 수행
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 성능
r2_test = r2_score(y_test, y_pred)
print("Best R2 Score:", r2_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300, 'reg_lambda': 0.1, 'subsample': 0.8}
Best RMSE Score: 1.2548255211489345
Best R2 Score: 0.34045355146651335


In [37]:
grid_search.best_params_

{'learning_rate': 0.05,
 'max_depth': 3,
 'n_estimators': 300,
 'reg_lambda': 0.1,
 'subsample': 0.8}

In [36]:
# 예측
# 학습 데이터에 대한 예측
y_pred_train = best_model.predict(X_train)
# 테스트 데이터에 대한 예측
y_pred_test = best_model.predict(X_test)

# 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 훈련 데이터 성능 지표 계산
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
# 테스트 데이터 성능 지표 계산
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
# 결과 출력
print("--- Training Set Performance ---")
print(f"MSE: {mse_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"R² Score: {r2_train:.4f}\n")
print("--- Test Set Performance ---")
print(f"MSE: {mse_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R² Score: {r2_test:.4f}")

--- Training Set Performance ---
MSE: 1.1183
RMSE: 1.0575
MAE: 0.8168
R² Score: 0.5101

--- Test Set Performance ---
MSE: 1.4853
RMSE: 1.2187
MAE: 0.9384
R² Score: 0.3405


### 학습률 조정

In [38]:
# 하이퍼파라미터 그리드 설정
param_grid = {
'max_depth': [3],       # 깊이 축소
'reg_lambda': [0.1],  # 정규화 강화
'learning_rate': [0.03, 0.05,],
'n_estimators': [300, 400, 500],
'subsample': [0.8]
}


# 그리드 서치 설정
grid_search = GridSearchCV(
    estimator=LGBM,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적 결과 출력
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", -grid_search.best_score_)


# 최적 모델로 예측 수행
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 성능
r2_test = r2_score(y_test, y_pred)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Parameters: {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 400, 'reg_lambda': 0.1, 'subsample': 0.8}
Best RMSE Score: 1.2547139779975744


In [39]:
# 예측
# 학습 데이터에 대한 예측
y_pred_train = best_model.predict(X_train)
# 테스트 데이터에 대한 예측
y_pred_test = best_model.predict(X_test)

# 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 훈련 데이터 성능 지표 계산
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
# 테스트 데이터 성능 지표 계산
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
# 결과 출력
print("--- Training Set Performance ---")
print(f"MSE: {mse_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"R² Score: {r2_train:.4f}\n")
print("--- Test Set Performance ---")
print(f"MSE: {mse_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R² Score: {r2_test:.4f}")

--- Training Set Performance ---
MSE: 1.1811
RMSE: 1.0868
MAE: 0.8391
R² Score: 0.4826

--- Test Set Performance ---
MSE: 1.4827
RMSE: 1.2177
MAE: 0.9383
R² Score: 0.3416


In [40]:
grid_search.best_params_

{'learning_rate': 0.03,
 'max_depth': 3,
 'n_estimators': 400,
 'reg_lambda': 0.1,
 'subsample': 0.8}

# > XGBoost

In [4]:
data = model_df.copy()

## 2. 모델 훈련

In [48]:
from xgboost import XGBRegressor

XGB_stop = XGBRegressor(
    eval_metric='rmse',
    early_stopping_rounds=10,
    random_state=42
)

# 조기중단을 위한 평가 세트 설정
evals = [(X_train, y_train), (X_valid, y_valid)]

XGB_stop.fit(X_train, y_train, eval_set = evals, verbose=True)

[0]	validation_0-rmse:1.34587	validation_1-rmse:1.44221
[1]	validation_0-rmse:1.23969	validation_1-rmse:1.38011
[2]	validation_0-rmse:1.16801	validation_1-rmse:1.34963
[3]	validation_0-rmse:1.11131	validation_1-rmse:1.32706
[4]	validation_0-rmse:1.06863	validation_1-rmse:1.31288
[5]	validation_0-rmse:1.02545	validation_1-rmse:1.30579
[6]	validation_0-rmse:0.99456	validation_1-rmse:1.30381
[7]	validation_0-rmse:0.97481	validation_1-rmse:1.30293


[8]	validation_0-rmse:0.95183	validation_1-rmse:1.30468
[9]	validation_0-rmse:0.92848	validation_1-rmse:1.30426
[10]	validation_0-rmse:0.90058	validation_1-rmse:1.30221
[11]	validation_0-rmse:0.88715	validation_1-rmse:1.30140
[12]	validation_0-rmse:0.87535	validation_1-rmse:1.30381
[13]	validation_0-rmse:0.86990	validation_1-rmse:1.30493
[14]	validation_0-rmse:0.85678	validation_1-rmse:1.30515
[15]	validation_0-rmse:0.84938	validation_1-rmse:1.30694
[16]	validation_0-rmse:0.84049	validation_1-rmse:1.30717
[17]	validation_0-rmse:0.83754	validation_1-rmse:1.30785
[18]	validation_0-rmse:0.81958	validation_1-rmse:1.30648
[19]	validation_0-rmse:0.80648	validation_1-rmse:1.30864
[20]	validation_0-rmse:0.79734	validation_1-rmse:1.30990


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=10,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

## 3. 예측 및 평가

In [49]:
# 예측
# 학습 데이터에 대한 예측
y_pred_train = XGB_stop.predict(X_train)
# 테스트 데이터에 대한 예측
y_pred_test = XGB_stop.predict(X_test)

# 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 훈련 데이터 성능 지표 계산
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
# 테스트 데이터 성능 지표 계산
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
# 결과 출력
print("--- Training Set Performance ---")
print(f"MSE: {mse_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"R² Score: {r2_train:.4f}\n")
print("--- Test Set Performance ---")
print(f"MSE: {mse_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R² Score: {r2_test:.4f}")

--- Training Set Performance ---
MSE: 0.7870
RMSE: 0.8872
MAE: 0.6917
R² Score: 0.6552

--- Test Set Performance ---
MSE: 1.5912
RMSE: 1.2614
MAE: 0.9604
R² Score: 0.2934


## 4. grid search

In [53]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
'max_depth': [3, 4],       # 깊이 축소
'reg_lambda': [0.1, 0.3],  # 정규화 강화
'learning_rate': [0.05, 0.07, 0.1], # 학습률 감소
'n_estimators': [300, 400, 500], # 트리 수 증가
'subsample': [0.8]
}


# 그리드 서치 설정
grid_search = GridSearchCV(
    estimator=XGB_stop,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train, eval_set = evals, verbose=True)

# 최적 결과 출력
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", -grid_search.best_score_)


# 최적 모델로 예측 수행
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 성능
r2_test = r2_score(y_test, y_pred)
print("Best R2 Score:", r2_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[0]	validation_0-rmse:1.47620	validation_1-rmse:1.51983
[1]	validation_0-rmse:1.44717	validation_1-rmse:1.49303
[2]	validation_0-rmse:1.42181	validation_1-rmse:1.46975
[3]	validation_0-rmse:1.40042	validation_1-rmse:1.45142
[4]	validation_0-rmse:1.38130	validation_1-rmse:1.43063
[5]	validation_0-rmse:1.36472	validation_1-rmse:1.41496
[6]	validation_0-rmse:1.35060	validation_1-rmse:1.39755
[7]	validation_0-rmse:1.33770	validation_1-rmse:1.38539
[8]	validation_0-rmse:1.32411	validation_1-rmse:1.37374
[9]	validation_0-rmse:1.31361	validation_1-rmse:1.36285
[10]	validation_0-rmse:1.30395	validation_1-rmse:1.35486
[11]	validation_0-rmse:1.29398	validation_1-rmse:1.34734
[12]	validation_0-rmse:1.28582	validation_1-rmse:1.33978
[13]	validation_0-rmse:1.27893	validation_1-rmse:1.33363
[14]	validation_0-rmse:1.27271	validation_1-rmse:1.32980
[15]	validation_0-rmse:1.26688	validation_1-rmse:1.32355
[16]	validation_0-rmse:1.26139	valid

In [54]:
# 예측
# 학습 데이터에 대한 예측
y_pred_train = best_model.predict(X_train)
# 테스트 데이터에 대한 예측
y_pred_test = best_model.predict(X_test)

# 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 훈련 데이터 성능 지표 계산
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
# 테스트 데이터 성능 지표 계산
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
# 결과 출력
print("--- Training Set Performance ---")
print(f"MSE: {mse_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"R² Score: {r2_train:.4f}\n")
print("--- Test Set Performance ---")
print(f"MSE: {mse_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R² Score: {r2_test:.4f}")

--- Training Set Performance ---
MSE: 1.3308
RMSE: 1.1536
MAE: 0.9041
R² Score: 0.4170

--- Test Set Performance ---
MSE: 1.5460
RMSE: 1.2434
MAE: 0.9561
R² Score: 0.3135


In [55]:
grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 300,
 'reg_lambda': 0.3,
 'subsample': 0.8}

### 학습률 조정

In [58]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
'max_depth': [3],       # 깊이 축소
'reg_lambda': [0.3],  # 정규화 강화
'learning_rate': [0.05, 0.09, 0.1], # 학습률 감소
'n_estimators': [300, 350, 400], # 트리 수 증가
'subsample': [0.8]
}


# 그리드 서치 설정
grid_search = GridSearchCV(
    estimator=XGB_stop,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train, eval_set = evals, verbose=True)

# 최적 결과 출력
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", -grid_search.best_score_)


# 최적 모델로 예측 수행
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 성능
r2_test = r2_score(y_test, y_pred)
print("Best R2 Score:", r2_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[0]	validation_0-rmse:1.47620	validation_1-rmse:1.51983
[1]	validation_0-rmse:1.44717	validation_1-rmse:1.49303
[2]	validation_0-rmse:1.42181	validation_1-rmse:1.46975
[3]	validation_0-rmse:1.40042	validation_1-rmse:1.45142
[4]	validation_0-rmse:1.38130	validation_1-rmse:1.43063
[5]	validation_0-rmse:1.36472	validation_1-rmse:1.41496
[6]	validation_0-rmse:1.35060	validation_1-rmse:1.39755
[7]	validation_0-rmse:1.33770	validation_1-rmse:1.38539
[8]	validation_0-rmse:1.32411	validation_1-rmse:1.37374
[9]	validation_0-rmse:1.31361	validation_1-rmse:1.36285
[10]	validation_0-rmse:1.30395	validation_1-rmse:1.35486
[11]	validation_0-rmse:1.29398	validation_1-rmse:1.34734
[12]	validation_0-rmse:1.28582	validation_1-rmse:1.33978
[13]	validation_0-rmse:1.27893	validation_1-rmse:1.33363
[14]	validation_0-rmse:1.27271	validation_1-rmse:1.32980
[15]	validation_0-rmse:1.26688	validation_1-rmse:1.32355
[16]	validation_0-rmse:1.26139	validat

In [59]:
data["PHI_BENEFIT"].std()

1.517057481202885